## 1. 주제 선정 이유

최근 코로나 바이러스 감염증-19(COVID-19, 이하 코로나19)의 확진자가 늘어났다가 줄어들었다가를 반복하고 있는 한편, 백신(예방접종) 투약이 이뤄지기 시작하였습니다.

이러한 속도로 보았을 때에, 집단 면역이 이루어질 때가 언제일지 궁금하였습니다.

물론, 일반적인 뉴스에서 찾아보는 정보로는 [11월에 우리나라에서 집단 면역이 이루어질 것](https://www.news1.kr/articles/?4285537)이라 예상하지만, 이를 스스로도 예상해보고 싶어서 이러한 주제를 선정하게 되었습니다.

국가 간의 외교, 바이러스의 변종 등의 요소는 선택한 데이터들에 들어있지 않기에 예상 결과가 일반적인 예상인 11월과는 조금 달라질 것입니다.

하지만 스스로 만들어보는 것에 의의를 두고, 열심히 만들어보겠습니다.

## 2. 주제 정의

공공데이터포털의 [공공데이터활용지원센터_코로나19 예방접종 통계 데이터 조회 서비스](https://www.data.go.kr/data/15077756/openapi.do)와 [보건복지부_코로나19 시·도발생_현황](https://www.data.go.kr/data/15043378/openapi.do) 를 통해 프로그램을 시작할 때마다 데이터를 불러와서, 항상 최신의 데이터를 사용하여 집단 면역이 이루어지는 시기를 예측합니다.

집단 면역의 기준은, [WHO](https://www.who.int/news-room/q-a-detail/herd-immunity-lockdowns-and-covid-19)에 따르면 일반적인 기준은 없고 질병마다 다르다고 하며, 그에 대한 예시로 홍역은 약 95%가 백신을, 소아마비는 약 80%가 백신을 맞아야 집단 면역이 이루어진다고 합니다.

[이 기사](https://www.nytimes.com/2020/12/24/health/herd-immunity-covid-coronavirus.html)를 참고하면, 코로나19의 집단 면역 기준은 바이러스 시작 쯤에 나온 예상 수치인 60~70%는 현 상황에서는 낮으며, 약 85% 이상이 되어야 한다고 전문가들은 주장합니다.

따라서, 아직 코로나19의 집단 면역 기준이 확실하지 않기 때문에, 예상 수치는 70%부터 5%씩 올려서 95%까지 확인해 볼 것입니다.

## 3. 인터넷을 통한 데이터의 획득

`requests` 모듈을 사용하여 API로부터 데이터를 가져올 것입니다.<br>
`api_key` 는 인코딩된 인증키이며, `api_key_decode` 는 디코딩된 인증키입니다.

In [ ]:
import requests
api_key = "myaCBEk1XX8eSIOiKYJOj5ksB%2BRl%2BxYCJVn2eaWckckSQnQt8gu6lU2HBqSP5Hsx6Gd8uXpXowCPRTjGJ5yDsw%3D%3D"

[보건복지부_코로나19 시·도발생_현황](https://www.data.go.kr/data/15043378/openapi.do)에서 데이터를 불러오고 있습니다.<br>
데이터의 형태는 XML이고, 필요없는 데이터도 몇 섞여있기에 가공할 필요가 있습니다.

해당 데이터의 요청변수는 아래와 같습니다.

| 키 이름 | 항목 크기 | 항목 구분 | 항목 설명 | 입력 예시 |
| :- | :-: | :-: | :- | :- |
| serviceKey | 4 | 필수 | 공공 데이터 포털에서 받은 인증키 | - |
| pageNo | 4 | 옵션 | 페이지 번호 | 1 |
| numOfRows | 4 | 옵션 | 한 페이지의 결과 수 | 10 |
| startCreateDt | 30 | 옵션 | 검색할 생성일 범위의 시작| 20210420 |
| endCreateDt | 15 | 옵션 | 검색할 생성일 범위의 종료 | 20210425 |

In [ ]:
def loadCovid19InfState(startDate, endDate, pageNo=1, numOfRows=10):
    url = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson"
    url = url + "?serviceKey=" + api_key
    
    if pageNo != None:
        url = url + "&pageNo=" + str(pageNo)
    if numOfRows != None:
        url = url + "&numOfRows=" + str(numOfRows)
    if startDate != None:
        url = url + "&startCreateDt=" + str(startDate)
    if endDate != None:
        url = url + "&endCreateDt=" + str(endDate)
    
    return requests.get(url)

In [ ]:
covid19InfStateResponse = loadCovid19InfState(startDate="20210401", endDate="20210410")
print(covid19InfStateResponse.text)

[공공데이터활용지원센터_코로나19 예방접종 통계 데이터 조회 서비스](https://www.data.go.kr/data/15077756/openapi.do)에서 데이터를 불러오고 있습니다.<br>
데이터의 형태는 JSON 이고, 필요없는 데이터도 몇 섞여있기에 가공할 필요가 있습니다.

현재 해당 API호출에서 `{"code":-4,"msg":"등록되지 않은 인증키 입니다."}` 라는 대답만 돌아오고 있습니다.<br>
여러 방법을 사용해보았으나 문제가 해결되지 않아, 문의 메일을 보내놓고 기다리고 있는 상태입니다.

대답을 받았고, 인코딩 및 디코딩의 문제였습니다. 현재 시점에는 해결되었습니다.

해당 데이터의 요청변수는 아래와 같습니다.

| 키 이름 | 항목 타입 | 항목 구분 | 항목 설명 | 입력 예시 |
| :- | :-: | :-: | :- | :- |
| serviceKey | string | 필수 | 공공 데이터 포털에서 받은 인증키 | - |
| page | int64 | 필수 | 페이지 번호 | 1 |
| perPage | int64 | 필수 | 한 페이지의 결과 수 | 10 |
| returnType | string | 선택 | 응답의 데이터 타입 (JSON or XML) (기본값 : JSON) | JSON |
| cond[baseDate::EQ] | string | 선택 | 요청 일자와 같은 일자의 통계를 가져옴 | 2021-04-25 00:00:00 |
| cond[baseDate::LT]] | string | 선택 | 요청 일자 이전의 통계를 가져옴 | 2021-04-25 00:00:00 |
| cond[baseDate::LTE] | string | 선택 | 요청 일자 이전이거나 같은 일자의 통계를 가져옴 | 2021-04-25 00:00:00 |
| cond[baseDate::GT] | string | 선택 | 요청 일자 이후의 통계를 가져옴 | 2021-04-25 00:00:00 |
| cond[baseDate::GTE] | string | 선택 | 요청 일자 이후거나 같은 일자의 통계를 가져옴 | 2021-04-25 00:00:00 |
| cond[sido::EQ] | string | 선택 | 지역명칭(시도명) | 서울특별시 |

In [ ]:
class DateQueryConditionType:
    EQUAL = 1 # 같을 때
    BEFORE = 2 # 이전
    BEFORE_EQUAL = 3 # 이전이거나 같을 때
    AFTER = 4 # 이후
    AFTER_EQUAL = 5 # 이후거나 같을 때
    
    @classmethod
    def toString(cls, condType):
        '''
        int -> string
        쿼리 사용을 위해 조건 값을 문자열로 변환한다.
        '''
        if condType == cls.EQUAL:
            return "cond[baseDate::EQ]"
        elif condType == cls.BEFORE:
            return "cond[baseDate::LT]"
        elif condType == cls.BEFORE_EQUAL:
            return "cond[baseDate::LTE]"
        elif condType == cls.AFTER:
            return "cond[baseDate::GT]"
        elif condType == cls.AFTER_EQUAL:
            return "cond[baseDate::GTE]"
        return "UNDEFINED"

class VaccineDateQueryCondition:
    def __init__(self, date, condType):
        self.date = date
        self.condType = condType

def loadVaccineState(page=1, perPage=10, dateConds=[], sido="전국"):
    if not type(dateConds) is list:
        print('Error: dateConds 가 리스트가 아닙니다.')
        return None
    
    url = "https://api.odcloud.kr/api/15077756/v1/vaccine-stat?"
    url = url + f"serviceKey={api_key}&page={page}&perPage={perPage}" # 기본 쿼리 데이터 추가
    
    for dateCond in dateConds: # 날짜 쿼리 데이터 추가
        condStr = DateQueryConditionType.toString(dateCond.condType)
        url = url + f"&{condStr}={dateCond.date}"
    
    if sido != None: # 시도 쿼리 데이터 추가
        url = url + f"&cond[sido::EQ]={sido}"
        
    return requests.get(url)

In [ ]:
dateConds = []
dateConds.append(VaccineDateQueryCondition("2021-04-25 00:00:00", DateQueryConditionType.BEFORE))
vaccineStateResponse = loadVaccineState(dateConds=dateConds, perPage=30)
print(vaccineStateResponse.text)

## 4. 분석을 위한 데이터의 가공

위에서 기술했듯이, 각 데이터에는 분석에 필요하지 않은 데이터가 있습니다.<br>
파이썬에서 쓰기 편하게 각 항목을 클래스를 이용하여 하나로 묶으면서 필요하지 않은 데이터를 버리도록 하겠습니다.

[보건복지부_코로나19 시·도발생_현황](https://www.data.go.kr/data/15043378/openapi.do)에서는 아이디 값, 기준일시, 총 확진자 수, 격리 중인 총 환자 수, 격리 해제된 총 환자 수, 총 사망자 수, 당일 지역발생 수를 하나의 클래스로 묶었습니다.

[공공데이터활용지원센터_코로나19 예방접종 통계 데이터 조회 서비스](https://www.data.go.kr/data/15077756/openapi.do)에서는 아이디 값, 기준일시, 당일 1차 접종자 수, 당일 2차 접종자 수, 총합 1차 접종자 수, 총합 2차 접종자 수를 하나의 클래스로 묶었습니다.

In [ ]:
class Covid19InfState:
    def __init__(self, ID, baseDate, infections, isolations, released, death, local_occur):
        self.ID = ID # 아이디
        self.baseDate = baseDate # 기준일시
        self.infections = infections # 총 확진자 수
        self.isolations = isolations # 격리 중인 총 환자 수
        self.released = released # 격리 해제된 총 환자 수
        self.death = death # 총 사망자 수
        self.local_occur = local_occur # 당일 지역발생 수
    
    def toString(self):
        return f'Covid19InfState:{{ID={self.ID}, baseDate={self.baseDate}, infections={self.infections}, isolations={self.isolations}, released={self.released}, death={self.death}, local_occur={self.local_occur}}}'

class VaccineState:
    def __init__(self, ID, baseDate, todayPrimaryVaccination, todaySecondaryVaccination, totalPrimaryVaccination, totalSecondaryVaccination):
        self.ID = ID # 아이디
        self.baseDate = baseDate # 기준일시
        self.todayPrimaryVaccination = todayPrimaryVaccination # 당일 1차 접종자 수
        self.todaySecondaryVaccination = todaySecondaryVaccination # 당일 2차 접종자 수
        self.totalPrimaryVaccination = totalPrimaryVaccination # 총합 1차 접종자 수
        self.totalSecondaryVaccination = totalSecondaryVaccination # 총합 2차 접종자 수
    
    def toString(self):
        return f'VaccineState:{{ID={self.ID}, baseDate={self.baseDate}, todayPV={self.todayPrimaryVaccination}, todaySV={self.todaySecondaryVaccination}, totalPV={self.totalPrimaryVaccination}, totalSV={self.totalSecondaryVaccination}}}'
    

코로나19 발생 현황 데이터를 위에서 만든 `Covid19InfState` 클래스에 맞춰서 넣기 위해 XML을 파싱하고, 객체를 생성하여 리스트에 넣습니다.

In [ ]:
import xml.etree.ElementTree as elemTree

covid19InfStateList = []

tree = elemTree.fromstring(covid19InfStateResponse.text) # XML
items = tree.find('./body/items')

for item in items.findall('item'):
    gubun = item.find('gubun').text
    if gubun == '합계':
        ID = item.find('seq').text
        baseDate = item.find('stdDay').text
        infections = item.find('defCnt').text
        isolations = item.find('isolIngCnt').text
        released = item.find('isolClearCnt').text
        death = item.find('deathCnt').text
        local_occur = item.find('localOccCnt').text
        
        newCovid19InfState = Covid19InfState(ID, baseDate, infections, isolations, released, death, local_occur)
        print(newCovid19InfState.toString())
        
        covid19InfStateList.append(newCovid19InfState)

코로나19 예방접종 현황 데이터를 위에서 만든 `VaccineState` 클래스에 맞춰서 넣기 위해 JSON을 파싱하고, 객체를 생성하여 리스트에 넣습니다.

In [ ]:
import json

vaccineStateList = []

jsonObject = json.loads(vaccineStateResponse.text) # JSON
dataList = jsonObject.get("data")

for idx, data in enumerate(dataList):
    ID = idx
    baseDate = data.get("baseDate")
    todayPV = data.get("firstCnt")
    todaySV = data.get("secondCnt")
    totalPV = data.get("totalFirstCnt")
    totalSV = data.get("totalSecondCnt")
    
    newVaccineState = VaccineState(ID, baseDate, todayPV, todaySV, totalPV, totalSV)
    print(newVaccineState.toString())
    
    vaccineStateList.append(newVaccineState)

## 7. 참고문헌

### 데이터 출처

- 코로나19 발생 현황 : https://www.data.go.kr/data/15043378/openapi.do
- 코로나19 예방접종 현황 : https://www.data.go.kr/data/15077756/openapi.do

### 데이터 가공

- XML Parsing : https://docs.python.org/3/library/xml.etree.elementtree.html
- JSON Parsing : https://docs.python.org/3/library/json.html

